In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import webbrowser
from googleapiclient.discovery import build

pd.options.display.max_colwidth = 200

api_key = "AIzaSyDvdfvwjvssvuzlN7u52biHmBJyStL34"

In [2]:
channel_url = "https://www.youtube.com/c/sentdex"

published_after = "2020-01-01"  # "%Y-%m-%d"

In [3]:
r = requests.get(channel_url)
soup = BeautifulSoup(r.text, "html.parser")
# channel_id = soup.find("meta", itemprop="channelId")['content']
channel_id = soup.find("link", itemprop="url")["href"].split("/")[-1]
upload_id = channel_id[0] + "U" + channel_id[2:]

yt = build("youtube", "v3", developerKey=api_key)

next_page_token = None
titles, urls, stats, dates = [], [], [], []

while True:
    res = yt.playlistItems().list(playlistId=upload_id, maxResults=50, part="snippet", pageToken=next_page_token).execute()
    items = res["items"]
    titles += [i["snippet"]["title"] for i in items]
    datex = [i["snippet"]["publishedAt"][:10] for i in items]
    dates += datex
    video_ids = [i["snippet"]["resourceId"]["videoId"] for i in items]
    urls += [f"https://www.youtube.com/watch?v={i}" for i in video_ids]
    stats += [i["statistics"] for i in yt.videos().list(id=",".join(video_ids), part="statistics").execute()["items"]]

    next_page_token = res.get("nextPageToken")
    if (datex[-1] < published_after) or next_page_token is None:
        break


def likes(i):
    try:
        return int(i["likeCount"])
    except:
        return 0


views = [int(i["viewCount"]) for i in stats]
likes = [likes(i) for i in stats]

data = {"Title": titles, "Views": views, "Likes": likes, "Date": dates, "URL": urls}
df = pd.DataFrame(data)
df.insert(3, "L/V", (df["Likes"] / df["Views"]).round(4))

In [4]:
# df = df[df["Date"] > published_after]

In [5]:
df0 = df.sort_values("Likes", ascending=False)
df0.head(10)

,Title,Views,Likes,L/V,Date,URL
49,Neural Networks from Scratch - P.1 Intro and Neuron Code,796561,22725,0.0285,2020-04-10,https://www.youtube.com/watch?v=Wo5dMEP_BbI
19,Playing a Neural Network's version of GTA V: GAN Theft Auto,699378,9062,0.0130,2021-06-15,https://www.youtube.com/watch?v=udPY5rQVoW0
48,Neural Networks from Scratch - P.2 Coding a Layer,231912,7796,0.0336,2020-04-16,https://www.youtube.com/watch?v=lGLto9Xd7bU
74,Introduction - Deep Learning and Neural Networks with Python and Pytorch p.1,367812,6890,0.0187,2019-09-21,https://www.youtube.com/watch?v=BzcBsTou0C0
47,Neural Networks from Scratch - P.3 The Dot Product,184546,6719,0.0364,2020-04-22,https://www.youtube.com/watch?v=tMrbN67U9d4
16,What Can Huge Neural Networks do?,134352,6396,0.0476,2021-08-04,https://www.youtube.com/watch?v=_z86t7LerrQ
39,Neural Networks from Scratch (NNFS) in Print!,132793,6078,0.0458,2020-10-26,https://www.youtube.com/watch?v=sNHiM0DoEAg
46,"Neural Networks from Scratch - P.4 Batches, Layers, and Objects",170246,5875,0.0345,2020-04-29,https://www.youtube.com/watch?v=TEWy9vZcxW4
61,Quantum Computer Programming w/ Qiskit,186748,5574,0.0298,2019-11-28,https://www.youtube.com/watch?v=aPCZcv-5qfA
42,Sentdex Channel Update,96410,5446,0.0565,2020-07-22,https://www.youtube.com/watch?v=kY2tKfsfyhU


In [6]:
df1 = df[df["Views"] > df["Views"].quantile(0.8)].sort_values("L/V", ascending=False)
df1.head(10)

,Title,Views,Likes,L/V,Date,URL
16,What Can Huge Neural Networks do?,134352,6396,0.0476,2021-08-04,https://www.youtube.com/watch?v=_z86t7LerrQ
39,Neural Networks from Scratch (NNFS) in Print!,132793,6078,0.0458,2020-10-26,https://www.youtube.com/watch?v=sNHiM0DoEAg
36,Cyber Python 2077 - Using computer vision to read and walk from Cyberpunk 2077 map,115157,4841,0.0420,2021-01-13,https://www.youtube.com/watch?v=dUU6ZsJlZKQ
47,Neural Networks from Scratch - P.3 The Dot Product,184546,6719,0.0364,2020-04-22,https://www.youtube.com/watch?v=tMrbN67U9d4
65,Brain Computer Interface w/ Python and OpenBCI for EEG data,133048,4672,0.0351,2019-10-25,https://www.youtube.com/watch?v=Dgo7F-lpyYE
46,"Neural Networks from Scratch - P.4 Batches, Layers, and Objects",170246,5875,0.0345,2020-04-29,https://www.youtube.com/watch?v=TEWy9vZcxW4
48,Neural Networks from Scratch - P.2 Coding a Layer,231912,7796,0.0336,2020-04-16,https://www.youtube.com/watch?v=lGLto9Xd7bU
45,Neural Networks from Scratch - P.5 Hidden Layer Activation Functions,162681,5357,0.0329,2020-05-13,https://www.youtube.com/watch?v=gmjzbpSVY1A
61,Quantum Computer Programming w/ Qiskit,186748,5574,0.0298,2019-11-28,https://www.youtube.com/watch?v=aPCZcv-5qfA
49,Neural Networks from Scratch - P.1 Intro and Neuron Code,796561,22725,0.0285,2020-04-10,https://www.youtube.com/watch?v=Wo5dMEP_BbI


In [7]:
df2 = df[(df["Views"].quantile(0.8) > df["Views"]) & (df["Views"] >= df["Views"].quantile(0.5))].sort_values("L/V", ascending=False)
df2.head(10)

,Title,Views,Likes,L/V,Date,URL
62,Thank You,55706,5150,0.0924,2019-11-25,https://www.youtube.com/watch?v=8Z8nkjmzC14
57,Neural Networks from Scratch announcement,81416,5211,0.0640,2020-01-06,https://www.youtube.com/watch?v=G7RDn8Xtf_Y
42,Sentdex Channel Update,96410,5446,0.0565,2020-07-22,https://www.youtube.com/watch?v=kY2tKfsfyhU
11,Github Copilot: Good or Bad?,45420,2018,0.0444,2021-11-12,https://www.youtube.com/watch?v=PDNt07UePvk
15,Robot Dogs: A Programmer's Best Friend,53046,2163,0.0408,2021-08-17,https://www.youtube.com/watch?v=U2nNI9Yp_0g
52,Who is the Sentdex Audience?,57603,2170,0.0377,2020-03-15,https://www.youtube.com/watch?v=j87KKBZbmqk
32,Checking out one of the world’s first NVIDIA DGX Station A100s,59959,2187,0.0365,2021-03-29,https://www.youtube.com/watch?v=0mAesfFt4us
43,AI-Based Game Engines with GameGAN research paper,49439,1790,0.0362,2020-05-21,https://www.youtube.com/watch?v=WFm1RZCVShU
13,Can we simulate a real robot?,66150,2267,0.0343,2021-10-10,https://www.youtube.com/watch?v=phTnbmXM06g
38,Neural Networks from Scratch - P.6 Softmax Activation,75868,2586,0.0341,2020-12-16,https://www.youtube.com/watch?v=omz_NdFgWyU


In [8]:
df3 = df[(df["Views"].quantile(0.5) > df["Views"]) & (df["Views"] >= df["Views"].quantile(0.2))].sort_values("L/V", ascending=False)
df3.head(10)

,Title,Views,Likes,L/V,Date,URL
77,"""Members only"" bug.",21649,1055,0.0487,2019-09-13,https://www.youtube.com/watch?v=LwNPWNCA7ZI
41,NNFS Update #2: Content done,28186,1346,0.0478,2020-09-25,https://www.youtube.com/watch?v=SN0VwEt2huo
87,Elon Musk TTS Neural Network...a work in progress.,30093,1334,0.0443,2019-06-26,https://www.youtube.com/watch?v=dI9M6JZR1lM
56,Sentdex Channel in 2020,38017,1551,0.0408,2020-01-17,https://www.youtube.com/watch?v=OwTN-18PIwE
31,Conversational AI w/ Jarvis - checking out the API,40064,1627,0.0406,2021-03-30,https://www.youtube.com/watch?v=fQzjgaKSrkc
28,"P.1 Chatbot with Mic input/Speaker output using Python, Jarvis, and DialoGPT",34155,1272,0.0372,2021-04-07,https://www.youtube.com/watch?v=CumHy6v7un0
7,Reinforcement Learning with Stable Baselines 3 - Introduction (P.1),21545,768,0.0356,2022-01-30,https://www.youtube.com/watch?v=XbWhJdQgi7E
92,Why I use a Standing Desk,26575,905,0.0341,2019-05-24,https://www.youtube.com/watch?v=rTyEMsD-NNM
76,Cloud vs Local GPU Hosting (what to use and when?),34709,1090,0.0314,2019-09-13,https://www.youtube.com/watch?v=62keA5pdykM
17,Neural Networks from Scratch - P.9 Introducing Optimization and derivatives,40734,1276,0.0313,2021-07-30,https://www.youtube.com/watch?v=txh3TQDwP1g


In [9]:
break

SyntaxError: 'break' outside loop (Temp/ipykernel_12556/668683560.py, line 1)

In [ ]:
for i in df0.URL[:5]:
    webbrowser.open_new_tab(i)

In [ ]:
for i in df1.URL[:5]:
    webbrowser.open_new_tab(i)

In [ ]:
for i in df2.URL[:5]:
    webbrowser.open_new_tab(i)

In [ ]:
for i in df3.URL[:5]:
    webbrowser.open_new_tab(i)